In [ ]:
from flask import Flask, request, send_file, render_template_string
import pandas as pd
import joblib
import pickle
from tensorflow.keras.models import load_model
import tempfile
import os

app = Flask(__name__)

# === Load model, scaler, features ===
model = load_model(r"C:\Users\HP\Desktop\final_project_again\flask_again\best_model.h5")
scaler = joblib.load(r"C:\Users\HP\Desktop\final_project_again\flask_again\scaler.pkl")
with open(r"C:\Users\HP\Desktop\final_project_again\flask_again\features.pkl", "rb") as f:
    selected_features = pickle.load(f)

# === HTML Template for upload ===
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Batch Prediction</title>
</head>
<body>
    <h2>Upload CSV File (Batch Prediction)</h2>
    <form action="/" method="post" enctype="multipart/form-data">
        <input type="file" name="file" accept=".csv" required>
        <button type="submit">Upload and Predict</button>
    </form>

    {% if download_link %}
        <h3>✅ Prediction Complete</h3>
        <a href="{{ download_link }}" download>Download Predictions CSV</a>
    {% endif %}

    {% if error %}
        <p style="color:red;">Error: {{ error }}</p>
    {% endif %}
</body>
</html>
"""

@app.route('/', methods=['GET', 'POST'])
def upload_predict():
    if request.method == 'POST':
        file = request.files.get('file')
        if not file or file.filename == '':
            return render_template_string(HTML_TEMPLATE, error="No file selected.")

        try:
            # Load CSV file
            df = pd.read_csv(file)

            if 'Gene ID' not in df.columns:
                return render_template_string(HTML_TEMPLATE, error="CSV must contain 'Gene ID' column.")

            # === Keep original Gene ID for final output ===
            gene_ids = df['Gene ID']

            # === Feature alignment ===
            input_df = df.reindex(columns=selected_features, fill_value=0)

            # === Predict ===
            scaled = scaler.transform(input_df)
            raw_preds = model.predict(scaled).flatten()
            predicted_classes = (raw_preds > 0.5).astype(int)

            # Add predictions and Gene ID back to result
            result_df = pd.DataFrame({
                'Gene ID': gene_ids,
                'Prediction': ["Positive" if p == 1 else "Negative" for p in predicted_classes]
            })

            # Count positive and negative
            positive_count = (predicted_classes == 1).sum()
            negative_count = (predicted_classes == 0).sum()

            # Add summary row
            summary_row = pd.DataFrame([["TOTAL", f"Positive: {positive_count}, Negative: {negative_count}"]],
                                       columns=result_df.columns)
            result_df = pd.concat([result_df, summary_row], ignore_index=True)

            # === Save to temporary file ===
            temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.csv', mode='w', newline='')
            result_df.to_csv(temp_file.name, index=False)
            temp_file.close()

            download_link = f"/download/{os.path.basename(temp_file.name)}"
            return render_template_string(HTML_TEMPLATE, download_link=download_link)

        except Exception as e:
            return render_template_string(HTML_TEMPLATE, error=str(e))

    return render_template_string(HTML_TEMPLATE)

# === Route to download file ===
@app.route('/download/<filename>')
def download_file(filename):
    file_path = os.path.join(tempfile.gettempdir(), filename)
    return send_file(file_path, as_attachment=True)

if __name__ == '__main__':
    app.run(debug=True)